# T1 - Installation and getting started

Installing and getting started with Covasim is quite simple. To install, just type `pip install covasim`. If it worked, you should be able to import Covasim with `import covasim as cv`.

The basic design philosphy of Covasim is: "**Common tasks should be simple**". For example:
- Defining parameters
- Running a simulation
- Plotting results

**Defining parameters**

Parameters are defined as a dictionary. The most common parameters to modify are the population size, the initial number of people infected, and the start and end dates of the simulation. We can define those as:

In [4]:
pars = dict(
    pop_size = 50000,
    pop_infected = 100,
    start_day = '2020-04-01',
    end_day = '2020-05-01'
)

**Running a simulation**

Running a simulation is really easy. In fact, it's just

In [5]:
import covasim as cv

sim = cv.Sim(pars)
results = sim.run()

Initializing sim with 50000 people for 30 days
  Running 2020-04-01 ( 0/30) (0.45 s)  ———————————————————— 3%
  Running 2020-04-02 ( 1/30) (0.45 s)  •——————————————————— 6%
  Running 2020-04-03 ( 2/30) (0.46 s)  •——————————————————— 10%
  Running 2020-04-04 ( 3/30) (0.47 s)  ••—————————————————— 13%
  Running 2020-04-05 ( 4/30) (0.47 s)  •••————————————————— 16%
  Running 2020-04-06 ( 5/30) (0.48 s)  •••————————————————— 19%
  Running 2020-04-07 ( 6/30) (0.48 s)  ••••———————————————— 23%
  Running 2020-04-08 ( 7/30) (0.49 s)  •••••——————————————— 26%
  Running 2020-04-09 ( 8/30) (0.50 s)  •••••——————————————— 29%
  Running 2020-04-10 ( 9/30) (0.50 s)  ••••••—————————————— 32%
  Running 2020-04-11 (10/30) (0.51 s)  •••••••————————————— 35%
  Running 2020-04-12 (11/30) (0.51 s)  •••••••————————————— 39%
  Running 2020-04-13 (12/30) (0.52 s)  ••••••••———————————— 42%
  Running 2020-04-14 (13/30) (0.53 s)  •••••••••——————————— 45%
  Running 2020-04-15 (14/30) (0.53 s)  •••••••••———————————

**Plotting a simulation**

Guess what? Plotting a simulation is easy too:

And that's it.